## Quality control of raw sequencing data

In [ ]:
# run fastp in default moder for 56 samples
fastp -i sample_R1.fq.gz -I sample_R2.fq.gz -o sample_R1_trimmed.fq.gz -O sample_R2_trimmed.fq.gz

# run fastqc for quality check
# all good after fastqc, continue with trimmed reads

# reindeerY1320 assembly

In [ ]:
# For 3 reindeer merge trimmed read 1s in one file, merge trimmed read 2s in one file
domFin_1.fq.gz
domFin_2.fq.gz

# Mapp the merged files to the female reindeer draft assembly
bwa mem -M -t 16 female_reindeer_draft_assembly.fasta domFin_1.fq.gz domFin_2.fq.gz > domFin_mapped2_female_reindeer_draft_assembly.sam 
samtools view -S -b domFin_mapped2_female_reindeer_draft_assembly.sam > domFin_mapped2_female_reindeer_draft_assembly.bam

# Collect unmapped reads
samtools view -u -f 12 -F 256 domFin_mapped2_female_reindeer_draft_assembly.bam > domFin_mapped2_female_reindeer_draft_assembly_unMappedPairs.bam

# Convert unmapped reads bam file to fastq
bedtools bamtofastq -i domFin_mapped2_female_reindeer_draft_assembly_unMappedPairs.bam -fq domFin_mapped2_female_reindeer_draft_assembly_unMappedPairs_read1.fq -fq2 domFin_mapped2_female_reindeer_draft_assembly_unMappedPairs_read2.fq

# Assemble the unmapped reads 
spades.py --careful --pe1-1 domFin_mapped2_female_reindeer_draft_assembly_unMappedPairs_read1.fq --pe1-2 domFin_mapped2_female_reindeer_draft_assembly_unMappedPairs_read2.fq -o domFin_unMappedPairsAssembly

# Continue with the contigs longer than 300 bp
domFin_unMappedPairsAssembly_300bp.fasta

# run the probabilistic model to define single copy regions of the chromosome as detailed in Felkel et al., 2019

# Placing reindeerY1320 contigs on OX460344.1

In [ ]:
### For blastn alignment with 95 perc identity
/usr/local/Cellar/blast/ncbi-blast-2.15.0+/bin/blastn -outfmt 6 -query /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/reindeerY1320.fasta -subject /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/newYref/OX460344.1.fasta -out /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/blastAlign/OX460344.1-reindeerY1320_pident95_tab.txt -perc_identity 95
# filter out the alignments shorter than 


### For MUMmer alignment and visualisaiton
# align and create alignment report
/usr/local/Cellar/MUMmer3.23/nucmer -maxmatch --prefix=OX460344.1-reindeerY1320 /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/OX460344.1.fasta /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/reindeerY1320.fasta 
/usr/local/Cellar/MUMmer3.23/show-coords -r -l -I -T -H /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/mummerAlign/OX460344.1-reindeerY1320.delta > /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/mummerAlign/OX460344.1-reindeerY1320.coords
/usr/local/Cellar/MUMmer3.23/dnadiff -d /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/mummerAlign/OX460344.1-reindeerY1320.delta
# for dot plot visualisation on https://dot.sandbox.bio/
python3 DotPrep.py --delta OX460344.1-reindeerY1320.delta --out /Volumes/elif_01/02_reindeer/reindeerSecondRound/analysis4review/alignment2newYchr/mummerAlign

# Phylogeny analysis on reindeerY1320

### Mapping and filtering of trimmed read of samples on reindeerY1320

In [ ]:
# Index the reference
bwa index reindeerY1320.fasta

# Mapp the trimmed reads from each sample
bwa aln -t 36 -n 0.02 -l 200 reindeerY1320.fasta sample1_R1_trimmed.fq.gz > sample1_1.sai
bwa aln -t 36 -n 0.02 -l 200 reindeerY1320.fasta sample1_R2_trimmed.fq.gz > sample1_2.sai
bwa sampe reindeerY1320.fasta sample1_1.sai sample1_2.sai sample1_R1_trimmed.fq.gz sample1_R2_trimmed.fq.gz > sample1.sam

# Process the mappings for each sample
# get mapped and properly paired reads
samtools view -h -b -S -F 4 -f 2 sample1.sam > sample1_mapped_pp.bam
# sort the reads
samtools sort sample1_mapped_pp.bam -o sample1_mapped_pp_sorted.bam
# remove duplicates
samtools rmdup -S sample1_mapped_pp_sorted.bam sample1_mapped_pp_sorted_rmdup.bam
# keep the reads having at least 20 mapping quality
samtools view -q 20 -b sample1_mapped_pp_sorted_rmdup.bam > sample1_mapped_pp_sorted_rmdup_mq20.bam

# merge the bam files of the samples if a sample has multiple lanes
samtools merge sample1_mapped_pp_sorted_rmdup_mq20_merged.bam sample1_1_mapped_pp_sorted_rmdup_mq20.bam sample1_2_mapped_pp_sorted_rmdup_mq20.bam sample1_3_mapped_pp_sorted_rmdup_mq20.bam

# arrange readgroup info in the bam file if needed
java -jar picard.jar AddOrReplaceReadGroups I=sample1_mapped_pp_sorted_rmdup_mq20_merged.bam O=sample1_mapped_pp_sorted_rmdup_mq20_merged_rg.bam RGID=sample_1 RGLB=sample_1 RGPL=ILLUMINA RGPU=sample_1 RGSM=sample_1

# index the final bam file
samtools index sample1_mapped_pp_sorted_rmdup_mq20_merged_rg.bam


### Y chromosomal variant ascertainment and filtering

In [ ]:
### 1. Variant ascertainment & filtering
#for each different depth group of Rangifer and for moose SNPs following steps applied

# freebayes calling/genotyping
freebayes -p 1 --genotype-qualities --fasta-reference reindeerY1320.fasta --bam-list samplesList.txt --vcf samples_freebayes_p1.vcf

# delete variants which does not show alternative call
vcftools --vcf samples_freebayes_p1.vcf --non-ref-ac-any 1 --recode --recode-INFO-all --out samples_freebayes_p1_noHomoRef

# remove complex variants
grep -v 'TYPE=.*complex.*' samples_freebayes_p1_noHomoRef.recode.vcf > samples_freebayes_p1_noHomoRef_noComplex.vcf

# remove multiallelic variants
bcftools view --max-alleles 2 samples_freebayes_p1_noHomoRef_noComplex.vcf > samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic.vcf

# bcftools normalisation
bcftools norm -a -f reindeerY1320.fasta samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic.vcf -o samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm.vcf

# filter for single copy Y reigon
vcftools --vcf samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm.vcf --bed reindeerY1320_scY.bed --recode --recode-INFO-all --out samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm_scY

# filter out variants in +- 50 contig ends 
vcftools --vcf samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm_scY.vcf --bed reindeerY1320_contigEnds50bpShorter.bed --recode --recode-INFO-all --out samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm_scY_contigendsShorter50bp

# get snps from the final variant list
bcftools view --types snps samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm_scY_contigendsShorter50bp.vcf > samples_freebayes_p1_noHomoRef_noComplex_noMultiAllelic_bcftoolsnorm_scY_contigendsShorter50bp_snps.vcf

### 2. variant genotyping & filtering
# merge and sort the SNPs detected in all different group in a bed file
bedtools sort -i mergedSNPlist.bed > mergedSNPlist_sorted.bed
bedtools merge -i mergedSNPlist_sorted > mergedSNPlist_sorted_merged.bed

# genotype these SNPs with freebayes
freebayes -p 1 --genotype-qualities --fasta-reference reindeerY1320.fasta --targets mergedSNPlist_sorted_merged.bed --bam-list allSamples.txt --vcf allSamples_genotyping_freebayes_p1.vcf --use-mapping-quality --min-mapping-quality 25 --min-base-quality 20

# repeat the filtering steps in the first part when needed


### 3. for furter filtering of the SNPs
# count the number of alternative and reference position of each sample for each variant, decide to threshıold for filtering
# estimate the mean coverages of the good quality variants in different sequencing depths, decide the thresholds
# further check all of the variants left in the list and filter out if any:
    # the assembly errors, 
    # normalization failure by bcftools
    # missing calls
    # multi-nucleotide polymorphisms
    # variants in repetitive sequences
    # variants determined in only one sample in heterozygous state
    # recurrent variants. 


### Mapping reindeerY1320 variants on OX460344.1

In [ ]:
# get flanking regions of the variants (+-30 the snp is in the middle)
bedtools getfasta -name -fi reindeerY1320.fasta -bed variantFlankingRegions.bed -fo variantFlankingRegions.fa

# map the flanking region fasta file to the new chromosome
bwa aln -t 4 -n 0.02 -l 200 OX460344.1.fasta variantFlankingRegions.fa > variantFlankingRegions_mapped2OX460344-1.sai
bwa samse OX460344.1.fasta variantFlankingRegions_mapped2OX460344-1.sai variantFlankingRegions.fa > variantFlankingRegions_mapped2OX460344-1.sam
samtools view -h -b -S variantFlankingRegions_mapped2OX460344-1.sam > variantFlankingRegions_mapped2OX460344-1.sam.bam


# mtDNA phylogeny analysis

In [ ]:
### 1. mapping and filtering of the trimmed read of samples on MZ353654 is performed 
# The same tools and parameters used as in Y chromosomal analysis

In [ ]:
### 2. create the multifasta alignment used in phylogeny

# create consensus fastas for the mappings
angsd -i sample1_mtMZ353654Mapped_pp_sorted_rmdup_mq20_merged_rg.bam -out sample1_mtMZ353654Mapped_pp_sorted_rmdup_mq20_merged_rg -doFasta 2 -doCounts 1 -setMinDepth 3 -minQ 20 -minMapQ 20
gunzip sample1_mtMZ353654Mapped_pp_sorted_rmdup_mq20_merged_rg.fa.gz

# merge the fasta files for all samples
# align the multifasta file
clustalo -i samples_mtMZ353654Mapped.fa -o samples_mtMZ353654Mapped_aligned.fa
